In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
import glob
import cv2
import csv

In [28]:
DATASET = 'data'

files = glob.glob('data/**/*.txt', recursive = True) #find all *.txt files

target_txt = "train_dataset.txt"

In [29]:
files_txt = glob.glob('data/**/*.txt', recursive = True)
files_png = glob.glob('data/**/*.png', recursive = True)

f_not_orig = []
f_orig = []
for f in files_txt:
    if "orig" not in f:
        f_not_orig.append(f)
    else:
        f_orig.append(f)
        
print(f"ORIGIN FILE: {len(f_orig)}\t NOT ORIGIN FILE: {len(f_not_orig)}")
print(f"TOTAL LEN: {len(files_txt)}")


im_mask = []
im_not_mask = []
for im in files_png:
    if "mask" in im:
        im_mask.append(im)
    else:
        im_not_mask.append(im)
print(f"MASK IMG: {len(im_mask)}\t NOT MASK IMG: {len(im_not_mask)}")
print(f"TOTAL LEN: {len(files_png)}")

ORIGIN FILE: 6074	 NOT ORIGIN FILE: 6074
TOTAL LEN: 12148
MASK IMG: 6074	 NOT MASK IMG: 6074
TOTAL LEN: 12148


In [20]:
# #ORIGIN FILES

# #if not os.path.exists(target_txt):
# if True:
#   fw = open("prova_dataset.txt",'w')
#   i = 0
#   for fi in files:
#     if "orig" in fi: #discard all *_orig.txt
#       f = open(fi, 'r')
#       file_text = f.read()
#       file_text = file_text.strip('\n') #remove possible newline character at the end of file
#       lines = file_text.split('\n')
#       file_name = os.path.join(fi.split("/")[-2],fi.split("/")[-1]) #last / and first part before the . with the name of the folder
#       for l in lines:
#         if float(l.split(" ")[6]) < 0.81:
#             label = l.split(" ")[0]
#             if int(label) == 6 or int(label) == 7:
#                 name = file_name[:-9].split(".")[0] + ".png"
#                 xc = float(l.split(" ")[1]) 
#                 yc = float(l.split(" ")[2])
#                 w = float(l.split(" ")[3])
#                 h = float(l.split(" ")[4])
#                 #take values as (x0,y0) and (x1,y1) for top-left and bottom-right points of BB
#                 x0 = float(xc - w / 2) 
#                 y0 = float(yc - h / 2) 
#                 x1 = float(xc + w / 2) 
#                 y1 = float(yc + h / 2)
#                 if int(label) == 6: #label 1 -> NO FALL
#                   fw.write(name + ',' + str(x0) + ',' + str(y0) + "," + str(x1) + "," + str(y1) + "," + str(1) + "\n")
#                 elif int(label) == 7: #label 2 -> FALL
#                   fw.write(name + ',' + str(x0) + ',' + str(y0) + "," + str(x1) + "," + str(y1) + "," + str(2) + "\n")
#   fw.close()
#   print("Finish")
# else:
#   print("File already created!")

Finish


In [30]:
#FILES NOT ORIG (normalized)
#if not os.path.exists(target_txt):
if True:
  fw = open(target_txt,'w')
  i = 0
  for fi in files:
    if "orig" not in fi: #discard all *_orig.txt
      f = open(fi, 'r')
      file_text = f.read()
      file_text = file_text.strip('\n') #remove possible newline character at the end of file
      lines = file_text.split('\n')
      file_name = os.path.join(fi.split("/")[-2],fi.split("/")[-1]) #last / and first part before the . with the name of the folder
      if len(lines) == 0:
            continue            
      for l in lines:
        if l == '':
            continue
        if float(l.split(" ")[6]) < 0.81:
            label = l.split(" ")[0]
            if int(label) == 6 or int(label) == 7:
                name = file_name.split(".")[0] + ".png"
                image = cv2.imread(os.path.join('data',name))
                height,width, _ = image.shape #values to denormalize data
                xc = float(l.split(" ")[1]) * width 
                yc = float(l.split(" ")[2]) * height
                w = float(l.split(" ")[3]) * width
                h = float(l.split(" ")[4]) * height
                #take values as (x0,y0) and (x1,y1) for top-left and bottom-right points of BB
                x0 = float(xc - w / 2) 
                y0 = float(yc - h / 2) 
                x1 = float(xc + w / 2) 
                y1 = float(yc + h / 2)

                if int(label) == 6: #label 1 -> NO FALL
                  fw.write(name + ',' + str(x0) + ',' + str(y0) + "," + str(x1) + "," + str(y1) + "," + str(1) + "\n")
                elif int(label) == 7: #label 2 -> FALL
                  fw.write(name + ',' + str(x0) + ',' + str(y0) + "," + str(x1) + "," + str(y1) + "," + str(2) + "\n")
  fw.close()
  print("Completed!")
else:
  print("File already created!")

Completed!


In [31]:
train_df = pd.read_csv(target_txt, sep=',', header=None, names=['img_path', 'x0', 'y0', 'x1', 'y1', 'label'])
train_df.head()

,img_path,x0,y0,x1,y1,label
0,19-12-09_1652_624622812/000_03_04.png,463.0,204.0,560.0,241.0,2
1,19-12-09_1652_624622812/000_03_04.png,64.0,470.0,210.0,540.0,2
2,19-12-09_1652_624622812/000_03_04.png,267.0,464.0,332.0,535.0,1
3,19-12-09_1652_624622812/000_03_04.png,577.0,184.0,610.0,294.0,1
4,19-12-09_1652_624622812/000_03_04.png,346.0,268.0,421.0,366.0,1


In [32]:
train_df['img_path'].nunique()

6071

In [33]:
train_df['label'].value_counts()

1    26125
2     7456
Name: label, dtype: int64

In [34]:
train_df.shape

(33581, 6)

In [35]:
df = train_df.groupby(['img_path','label']).size().reset_index(name='counts')
df

,img_path,label,counts
0,19-12-05_1733_281439187/000_02_00.png,1,4
1,19-12-05_1733_281439187/000_03_00.png,1,3
2,19-12-05_1733_281439187/000_03_00.png,2,3
3,19-12-05_1733_281439187/000_03_01.png,1,2
4,19-12-05_1733_281439187/000_03_01.png,2,3
...,...,...,...
8308,19-12-10_1334_699107828/000_03_00.png,2,2
8309,19-12-10_1334_699107828/000_03_01.png,1,3
8310,19-12-10_1334_699107828/000_03_01.png,2,2
8311,19-12-10_1334_699107828/000_04_00.png,1,5


In [36]:
img = df['img_path']
not_dup = df[~img.isin(img[img.duplicated()])]
not_dup

,img_path,label,counts
0,19-12-05_1733_281439187/000_02_00.png,1,4
11,19-12-05_1733_281439187/000_04_00.png,1,1
12,19-12-05_1733_281439187/000_05_00.png,1,3
15,19-12-05_1733_281439187/000_05_02.png,1,3
16,19-12-05_1733_281439187/000_05_03.png,2,3
...,...,...,...
8302,19-12-10_1334_699107828/000_01_02.png,1,4
8303,19-12-10_1334_699107828/000_01_03.png,1,5
8304,19-12-10_1334_699107828/000_01_04.png,1,4
8305,19-12-10_1334_699107828/000_02_00.png,1,3


In [37]:
not_dup['label'].value_counts()

1    3441
2     388
Name: label, dtype: int64